# Flight Status Predictor - EDA

In [152]:
# Import Libraries

import pandas as pd
import numpy as np
import plotly as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import seaborn as sns

pd.options.display.max_columns = 50


In [153]:
# Import data

df = pd.read_csv('Flight_On_Time_Data.csv')

## Organize DataFrame

In [154]:
df.head()

,Carrier_Name,Year,Quarter,Month,Month_Day,Week_Day,Flight_Date,Airline_DOT_ID,Tail_Number,Flight_Number,Origin_Airport_ID,Origin_City_Market_ID,Origin_IATA_Code,Origin_City_State,Destination_Airport_ID,Destination_City_Market_ID,Destination_IATA_Code,Destination_City_State,Dep_Time_Block_Group,Scheduled_Departure_Time,Actual_Dep_Time,Dep_Time_Offset,Dep_Delay_Minutes,Dep_Delay_At_Least_15_Minutes,Scheduled_Arrival_Time,Arr_Time_Block_Group,Actual_Arr_Time,Arr_Time_Offset,Arr_Delay_Minutes,Arr_Delay_At_Least_15_Minutes,Flight_Cancelled,Flight_Diverted,Scheduled_Gate_to_Gate_Time,Actual_Gate_to_Gate_Time,Time_Airborne,Number_of_Flights,Distance_Miles,Distance_Group,Carrier_Delay,Weather_Delay,National_Aviation_System_Delay,Security_Delay,Late_Aircraft_Delay
0,United Air Lines Inc.,2023,2,Apr,26,Wednesday,2023-04-26,19977,N851UA,1788,11298,30194,DFW,"Dallas/Fort Worth, TX",11618,31703,EWR,"Newark, NJ",Evening,1926,1933,7,7,0,2359,Night,12,13,13,0,0,0,213,219,162,1,1372,6,NaN,NaN,NaN,NaN,NaN
1,United Air Lines Inc.,2023,2,Apr,26,Wednesday,2023-04-26,19977,N37530,1787,11292,30325,DEN,"Denver, CO",13871,33316,OMA,"Omaha, NE",Evening,2051,2130,39,39,1,2316,Night,3,47,47,1,0,0,85,93,65,1,472,2,39.0,0.0,8.0,0.0,0.0
2,United Air Lines Inc.,2023,2,Apr,26,Wednesday,2023-04-26,19977,N78509,1786,12892,32575,LAX,"Los Angeles, CA",11042,30647,CLE,"Cleveland, OH",Night,2328,2319,-9,0,0,701,Early Morning,650,-11,0,0,0,0,273,271,249,1,2052,9,NaN,NaN,NaN,NaN,NaN
3,United Air Lines Inc.,2023,2,Apr,26,Wednesday,2023-04-26,19977,N47280,1785,10721,30721,BOS,"Boston, MA",12264,30852,IAD,"Washington, DC",Early Morning,600,557,-3,0,0,735,Early Morning,724,-11,0,0,0,0,95,87,69,1,413,2,NaN,NaN,NaN,NaN,NaN
4,United Air Lines Inc.,2023,2,Apr,26,Wednesday,2023-04-26,19977,N826UA,1784,14747,30559,SEA,"Seattle, WA",11292,30325,DEN,"Denver, CO",Morning,1116,1113,-3,0,0,1459,Early Afternoon,1453,-6,0,0,0,0,163,160,129,1,1024,5,NaN,NaN,NaN,NaN,NaN


In [155]:
# Dropping redundant or extraneous columns and 
# renaming target variable to Delayed15

df = df[['Carrier_Name', 
        #'Year', 'Quarter', 
        'Month', 
        #'Month_Day', 
        'Week_Day',
        #'Flight_Date', 'Airline_DOT_ID', 
        'Tail_Number', 
        #'Flight_Number',
        #'Origin_Airport_ID', 'Origin_City_Market_ID', 
        'Origin_IATA_Code',
        #'Origin_City_State', 'Destination_Airport_ID',
        #'Destination_City_Market_ID', 
        'Destination_IATA_Code',
        #'Destination_City_State', 
        'Dep_Time_Block_Group',
        'Scheduled_Departure_Time', 
        #'Actual_Dep_Time', 'Dep_Time_Offset',
        #'Dep_Delay_Minutes', 'Dep_Delay_At_Least_15_Minutes',
        'Scheduled_Arrival_Time', 
        'Arr_Time_Block_Group', #'Actual_Arr_Time',
        #'Arr_Time_Offset', 'Arr_Delay_Minutes', 
        'Arr_Delay_At_Least_15_Minutes',
        #'Flight_Cancelled', 'Flight_Diverted', 'Scheduled_Gate_to_Gate_Time',
        #'Actual_Gate_to_Gate_Time', 'Time_Airborne', 'Number_of_Flights',
        'Distance_Miles', 
        'Distance_Group', 
        #'Carrier_Delay', 'Weather_Delay',
        #'National_Aviation_System_Delay', 'Security_Delay',
        #'Late_Aircraft_Delay'
]].copy()

df.rename(columns={'Arr_Delay_At_Least_15_Minutes':'Delayed','Origin_IATA_Code':'Origin','Destination_IATA_Code':'Destination'}, inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6777978 entries, 0 to 6777977
Data columns (total 13 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   Carrier_Name              object
 1   Month                     object
 2   Week_Day                  object
 3   Tail_Number               object
 4   Origin                    object
 5   Destination               object
 6   Dep_Time_Block_Group      object
 7   Scheduled_Departure_Time  int64 
 8   Scheduled_Arrival_Time    int64 
 9   Arr_Time_Block_Group      object
 10  Delayed                   int64 
 11  Distance_Miles            int64 
 12  Distance_Group            int64 
dtypes: int64(5), object(8)
memory usage: 672.3+ MB


In [164]:
# Grouping numerical columns and
# Moving the target variable to be the right-most column

# Columns to move
cols_to_move = [
                'Distance_Miles',
                'Scheduled_Departure_Time', 
                'Scheduled_Arrival_Time' 
                ]

last_col = 'Delayed'

# Extracting the columns to move
columns_to_move = df[cols_to_move]

# Dropping these columns from the DataFrame
df.drop(columns=cols_to_move, inplace=True)

# Extracting the last column
last_column = df.pop(last_col)

# Reinserting the numerical columns
df = pd.concat([df, columns_to_move], axis=1)

# Reinserting the last column
df[last_col] = last_column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6777978 entries, 0 to 6777977
Data columns (total 13 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   Carrier_Name              object
 1   Month                     object
 2   Week_Day                  object
 3   Tail_Number               object
 4   Origin                    object
 5   Destination               object
 6   Dep_Time_Block_Group      object
 7   Arr_Time_Block_Group      object
 8   Distance_Group            int64 
 9   Distance_Miles            int64 
 10  Scheduled_Departure_Time  int64 
 11  Scheduled_Arrival_Time    int64 
 12  Delayed                   int64 
dtypes: int64(5), object(8)
memory usage: 672.3+ MB


## Delayed Flights Analysis 

### Total Number of Delayed Flights from <u>01 Apr 2023</u> to <u>31 Mar 2024</u>

In [157]:
# Extract total number of flights
num_flights = df['Delayed'].count()

# Extract total number of on time and delayed flights
on_time, delayed = df['Delayed'].value_counts()

# Display total number of flights, delayed flights, and percentage delayed flights
print(f'Total Number of Flights = {num_flights:,}')
print(f'Number of Delayed Flights = {delayed:,}')
print(f'Percentage of Delayed Flights = {(delayed/num_flights)*100:.3} %')

Total Number of Flights = 6,777,978
Number of Delayed Flights = 1,371,451
Percentage of Delayed Flights = 20.2 %


In [158]:
# Visualizing total delayed flight stats

# Data and Labels
data = [on_time, delayed]
keys = ['On Time', 'Delayed']

# Pie Chart
fig_pie = go.Figure(data=[go.Pie(
    values=data, 
    labels=keys, 
    pull=[0,0.1],
    rotation = 73,
    textinfo='label+percent',
    insidetextorientation='horizontal'
    )])

# Adjusting Figure Size
fig_pie.update_layout(
    width=500, 
    height=500,
    title_text='Percentage of Delayed Flights',
    title_x=0.5,
    showlegend=False
    )

fig_pie.show()

### *** Below I chose to keep the Month and Weekday variables as I wanted to see what percentage of delayed flights happen in each month and day. I wasn't sure how to do that had I kept the Flight_Date variable. *** 

In [160]:
# Define categorical variables
categorical_vars = [
    ('Carrier_Name', 'Carrier Name'),
    ('Month', 'Month'),
    ('Week_Day', 'Week Day'),
    ('Tail_Number', 'Tail Number'),
    ('Origin', 'Origin'),
    ('Destination', 'Destination'),
    ('Dep_Time_Block_Group', 'Departure Time Block Group'),
    ('Arr_Time_Block_Group', 'Arrival Time Block Group')
]

# Initialize subplot figure with two columns of four
fig = sp.make_subplots(rows=4, cols=2, subplot_titles=[title for _, title in categorical_vars])

# Create bar charts for each categorical variable
for i, (var, title) in enumerate(categorical_vars, start=1):
    if var in ['Tail_Number', 'Origin', 'Destination', 'Carrier_Name']:
        # Limit top 10 for Tail_Number, Origin, and Destination
        top_10 = df[df['Delayed'] == 1][var].value_counts().head(10).sort_values(ascending=False)
        labels = top_10.index
        values = (top_10.values / delayed) * 100
    elif var == 'Month':
        # Sort month in order from January to December
        month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        month_counts = df[df['Delayed'] == 1]['Month'].value_counts(normalize=True).loc[month_order]
        labels = month_counts.index
        values = month_counts.values * 100
    elif var == 'Week_Day':
        # Sort week day from Sunday to Saturday
        weekday_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
        weekday_counts = df[df['Delayed'] == 1]['Week_Day'].value_counts(normalize=True).loc[weekday_order]
        labels = weekday_counts.index
        values = weekday_counts.values * 100
    elif var == 'Dep_Time_Block_Group':
        # Sort time block group from Early Morning to Night
        dep_block_order = ['Early Morning', 'Morning', 'Early Afternoon', 'Afternoon', 'Evening', 'Night']
        dep_block_counts = df[df['Delayed'] == 1]['Dep_Time_Block_Group'].value_counts(normalize=True).loc[dep_block_order]
        labels = dep_block_counts.index
        values = dep_block_counts.values * 100
    else:
        # Sort time block group from Early Morning to Night
        arr_block_order = ['Early Morning', 'Morning', 'Early Afternoon', 'Afternoon', 'Evening', 'Night']
        arr_block_counts = df[df['Delayed'] == 1]['Arr_Time_Block_Group'].value_counts(normalize=True).loc[arr_block_order]
        labels = arr_block_counts.index
        values = arr_block_counts.values * 100
    
    # Create bar chart
    bar = px.bar(x=labels, y=values, labels={'x': title, 'y': 'Percentage of Delayed Flights (%)'})

    # Add bar chart to subplot
    for trace in bar.data:
        fig.add_trace(trace, row=(i-1) // 2 + 1, col=(i-1) % 2 + 1)

# Update layout
fig.update_layout(height=1200, width=800, title_text='Percentage of Delayed Flights by Feature', showlegend=False)

fig.show()